# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 150 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel150', 'models','t_dGP_train')

Linesearch     25;  Value 2.998155e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 2.03404957, MSLL 3.0968, NLPD 5.9453


### 1.1.4 Find multi q

In [6]:
criterion = 'TERBCM';

In [7]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.2052
   2.2085
   2.2050
   2.2035
   2.2071
   2.2024
   2.2047
   2.2034
   2.2083
   2.2013
   2.2092
   2.2032
   2.2031
   2.2061
   2.2035
   2.2038
   2.2090
   2.2065
   2.2087
   2.2096
   2.2072
   2.2043
   2.2064
   2.2012
   2.2038
   2.2060
   2.2010
   2.2017
   2.2035
   2.2057
   2.2005
   2.2021
   2.2045
   2.2053
   2.2027
   2.2052
   2.2038
   2.2028
   2.2034
   2.2008
   2.2039
   2.2070
   2.2025
   2.2036
   2.2071
   2.2057
   2.2031
   2.2031
   2.2057
   2.2011
   2.2024
   2.2031
   2.2032
   2.2006
   2.2019
   2.2032
   2.2011
   2.2038
   2.2062
   2.2081
   2.2043
   2.2058
   2.2022
   2.2084
   2.2100
   2.2097
   2.2093
   2.2018
   2.2068
   2.2017
   2.2018
   2.2035
   2.2065
   2.2026
   2.2087
   2.2021
   2.2017
   2.2077
   2.2002
   2.2011
   2.2028
   2.2054
   2.2093
   2.2097
   2.2013
   2.2004
   2.2002
   2.2064
   2.2015
   2.2054
   2.2078
   2.2079
   2.2079
   2.2044
   2.2048
   2.2068
   2.2037
   2.2055


In [8]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.40983
   0.40920
   0.40111
   0.40694
   0.40646
   0.40831
   0.40478
   0.40519
   0.40558
   0.40033
   0.40031
   0.40612
   0.40458
   0.40268
   0.40278
   0.40488
   0.40761
   0.40927
   0.40071
   0.40549
   0.40210
   0.40900
   0.40229
   0.40314
   0.40016
   0.40078
   0.40715
   0.40389
   0.40396
   0.40643
   0.40808
   0.40636
   0.40551
   0.40789
   0.40769
   0.40230
   0.40281
   0.40608
   0.40351
   0.40181
   0.40916
   0.40812
   0.40106
   0.40519
   0.40573
   0.40650
   0.40627
   0.40055
   0.40678
   0.40243
   0.40380
   0.40738
   0.40651
   0.40439
   0.40494
   0.40125
   0.40187
   0.40865
   0.40733
   0.40314
   0.40666
   0.40499
   0.40186
   0.40906
   0.40099
   0.40436
   0.40052
   0.40100
   0.40002
   0.40153
   0.40514
   0.40255
   0.40873
   0.40474
   0.40814
   0.40480
   0.40122
   0.40536
   0.40569
   0.40220
   0.40014
   0.40021
   0.40347
   0.40100
   0.40540
   0.40886
   0.40270
   0.40136
   0.40392
 

## 1.2 GRBCM vs TEGRBCM

In [9]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 2.01399819, MSLL 0.6571, NLPD 3.5055


### 1.2.4 Find multi q

In [10]:
criterion = 'TEGRBCM';

In [11]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0017
   2.0020
   2.0045
   2.0023
   2.0002
   2.0032
   2.0030
   2.0069
   2.0061
   2.0068
   2.0097
   2.0016
   2.0057
   2.0037
   2.0038
   2.0037
   2.0088
   2.0079
   2.0051
   2.0068
   2.0080
   2.0046
   2.0006
   2.0061
   2.0015
   2.0020
   2.0044
   2.0091
   2.0055
   2.0068
   2.0039
   2.0044
   2.0030
   2.0088
   2.0014
   2.0080
   2.0088
   2.0034
   2.0028
   2.0072
   2.0025
   2.0085
   2.0052
   2.0067
   2.0098
   2.0013
   2.0012
   2.0067
   2.0037
   2.0079
   2.0008
   2.0070
   2.0086
   2.0005
   2.0036
   2.0093
   2.0064
   2.0005
   2.0064
   2.0048
   2.0083
   2.0064
   2.0015
   2.0084
   2.0009
   2.0039
   2.0072
   2.0002
   2.0055
   2.0002
   2.0097
   2.0021
   2.0001
   2.0049
   2.0056
   2.0083
   2.0061
   2.0052
   2.0058
   2.0051
   2.0046
   2.0089
   2.0070
   2.0038
   2.0032
   2.0084
   2.0065
   2.0026
   2.0039
   2.0019
   2.0035
   2.0086
   2.0052
   2.0034
   2.0029
   2.0037
   2.0013
   2.0032


In [12]:
printf('Optimizing q\r\n');
iqs = 0.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.20541
   0.20681
   0.20908
   0.20200
   0.20022
   0.20182
   0.20082
   0.20482
   0.20812
   0.20058
   0.20121
   0.20705
   0.20145
   0.20030
   0.20850
   0.20688
   0.20706
   0.20002
   0.20952
   0.20527
   0.20229
   0.20968
   0.20075
   0.20075
   0.20931
   0.20225
   0.20940
   0.20801
   0.20963
   0.20078
   0.20739
   0.20540
   0.20238
   0.20704
   0.20688
   0.20946
   0.20620
   0.20665
   0.20174
   0.20557
   0.20188
   0.20108
   0.20906
   0.20124
   0.20065
   0.20603
   0.20826
   0.20372
   0.20137
   0.20583
   0.20570
   0.20989
   0.20293
   0.20238
   0.20117
   0.20365
   0.20687
   0.20314
   0.20522
   0.20817
   0.20894
   0.20217
   0.20380
   0.20017
   0.20346
   0.20469
   0.20334
   0.20406
   0.20295
   0.20945
   0.20182
   0.20235
   0.20835
   0.20698
   0.20001
   0.20169
   0.20125
   0.20430
   0.20360
   0.20937
   0.20561
   0.20875
   0.20413
   0.20142
   0.20703
   0.20027
   0.20070
   0.20607
   0.20627
 

## 1.3 GPoE vs TEGPoE

In [13]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 1.63538489, MSLL 3.4327, NLPD 6.2811


### 1.3.4 Find multi q

In [14]:
criterion = 'TEGPoE';

In [15]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0019
   2.0027
   2.0048
   2.0039
   2.0073
   2.0030
   2.0077
   2.0062
   2.0011
   2.0028
   2.0087
   2.0000
   2.0001
   2.0054
   2.0083
   2.0004
   2.0052
   2.0039
   2.0031
   2.0045
   2.0100
   2.0001
   2.0069
   2.0075
   2.0042
   2.0019
   2.0067
   2.0056
   2.0072
   2.0054
   2.0018
   2.0097
   2.0002
   2.0089
   2.0062
   2.0021
   2.0000
   2.0068
   2.0045
   2.0035
   2.0023
   2.0053
   2.0068
   2.0028
   2.0015
   2.0005
   2.0089
   2.0062
   2.0082
   2.0032
   2.0007
   2.0083
   2.0078
   2.0061
   2.0092
   2.0073
   2.0050
   2.0092
   2.0032
   2.0052
   2.0038
   2.0037
   2.0014
   2.0099
   2.0003
   2.0079
   2.0035
   2.0024
   2.0038
   2.0013
   2.0070
   2.0013
   2.0086
   2.0042
   2.0032
   2.0098
   2.0096
   2.0012
   2.0022
   2.0094
   2.0062
   2.0092
   2.0019
   2.0049
   2.0043
   2.0082
   2.0099
   2.0070
   2.0001
   2.0008
   2.0039
   2.0067
   2.0099
   2.0089
   2.0052
   2.0008
   2.0022
   2.0030


In [16]:
printf('Optimizing q\r\n');
iqs = 1.8 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   1.8012
   1.8035
   1.8091
   1.8099
   1.8088
   1.8065
   1.8088
   1.8025
   1.8085
   1.8071
   1.8035
   1.8035
   1.8084
   1.8013
   1.8057
   1.8087
   1.8090
   1.8005
   1.8091
   1.8079
   1.8080
   1.8070
   1.8019
   1.8086
   1.8060
   1.8032
   1.8059
   1.8024
   1.8068
   1.8091
   1.8071
   1.8029
   1.8011
   1.8037
   1.8003
   1.8086
   1.8049
   1.8082
   1.8037
   1.8019
   1.8053
   1.8090
   1.8020
   1.8067
   1.8054
   1.8061
   1.8080
   1.8086
   1.8010
   1.8063
   1.8066
   1.8087
   1.8069
   1.8099
   1.8066
   1.8057
   1.8024
   1.8016
   1.8090
   1.8097
   1.8045
   1.8040
   1.8078
   1.8077
   1.8006
   1.8053
   1.8055
   1.8060
   1.8001
   1.8043
   1.8025
   1.8087
   1.8024
   1.8075
   1.8094
   1.8075
   1.8073
   1.8080
   1.8089
   1.8022
   1.8090
   1.8062
   1.8015
   1.8046
   1.8000
   1.8088
   1.8099
   1.8006
   1.8037
   1.8065
   1.8066
   1.8045
   1.8100
   1.8042
   1.8029
   1.8029
   1.8096
   1.8051
